## 步骤一：导入数据集

In [1]:
from sklearn.cross_validation import train_test_split
#from keras.utils import np_utils
import numpy as np
import os
from glob import glob

# 定义函数来加载train，test和validation数据集
def load_dataset(path):
    list = os.listdir(path)
    files = []
    targets = []
    for i in range(0,len(list)):
        filepath = os.path.join(path,list[i])
        if os.path.isfile(filepath):
            files.append(filepath)
            if(list[i].find('dog')>=0):
                targets.append([1,0])
            else:
                targets.append([0,1])
    return files,targets


# 加载train，valid数据集
#train_files, train_targets = load_dataset('data/train')
#valid_files, valid_targets = load_dataset('data/valid')



# 将数据集拆分成训练集，校验集,测试集
#tmp_files, test_files, tmp_targets, test_targets = train_test_split(data_files, data_targets, test_size = 0.2,random_state=10)
#train_files, valid_files, train_targets, valid_targets = train_test_split(tmp_files, tmp_targets, test_size = 0.25,random_state=10)

# 打印数据统计描述
#a = np.array(train_targets)
#print('训练集中有{}只狗，{}只猫'.format(sum(a[:,0]),sum(a[:,1])))

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 步骤二：数据预处理

In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # 用PIL加载RGB图像为PIL.Image.Image类型
    img = image.load_img(img_path, target_size=(224, 224))
    # 将PIL.Image.Image类型转化为格式为(224, 224, 3)的3维张量
    x = image.img_to_array(img)
    # 将3维张量转化为格式为(1, 224, 224, 3)的4维张量并返回
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

d:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 步骤三：使用迁移训练
### 提取特征

In [3]:
from keras.preprocessing import image

#图片数组生成器
def generate_arrays_from_path(files):
    while 1:
        for i in range(0,len(files)):
            X = preprocess_input(path_to_tensor(files[i]).astype('float32')/255)
                
            yield X


In [4]:
imageDataGen = image.ImageDataGenerator(
    rescale = 1.0/255,
#    rotation_range=30,
#    width_shift_range=0.3,
#    height_shift_range=0.3,
#    zoom_range=0.5,
#    channel_shift_range=10,
#    preprocessing_function = preprocess_input
)

In [32]:
img = image.load_img('train2/cat.0.jpg') 
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in imageDataGen.flow(x,batch_size=1,save_to_dir='preview',save_prefix='test3', save_format='jpg'):
    i += 1
    if i > 20:
        break  # 否则生成器会退出循环

In [5]:
train_generator = imageDataGen.flow_from_directory(
        'data/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=False) 

valid_generator = imageDataGen.flow_from_directory(
        'data/valid',
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=False) 

Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
from keras.applications.vgg16 import VGG16

from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

'''
bottleneck_features_train = model.predict_generator(generate_arrays_from_path(train_files),15000,verbose=2)
bottleneck_features_valid = model.predict_generator(generate_arrays_from_path(valid_files),5000,verbose=2)
bottleneck_features_test = model.predict_generator(generate_arrays_from_path(test_files),5000,verbose=2)
'''

bottleneck_features_train = model.predict_generator(train_generator,1000)
bottleneck_features_valid = model.predict_generator(valid_generator,400)
#bottleneck_features_test = model.predict_generator(generate_arrays_from_path(test_files),300,verbose=2)

np.save(open('bottleneck_features_train.npy','wb'),bottleneck_features_train)
np.save(open('bottleneck_features_valid.npy','wb'),bottleneck_features_valid)
#np.save(open('bottleneck_features_test.npy','wb'),bottleneck_features_test)

## 步骤四：训练模型
### 搭建VGG迁移模型

In [ ]:
train_VGG16 = np.load('bottleneck_features_train.npy')
valid_VGG16 = np.load('bottleneck_features_valid.npy')
#test_VGG16 = np.load('bottleneck_features_test.npy')


train_targets = np.array([0,1] * 500 + [1,0] * 500)
valid_targets = np.array([0,1] * 200 + [1,0] * 200)


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dropout(0.5))
VGG16_model.add(Dense(2, activation='softmax'))

VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
print(len(test_targets))

In [ ]:
from keras.callbacks import ModelCheckpoint  
## 训练模型

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, np.array(train_targets), 
          validation_data=(valid_VGG16, np.array(valid_targets)),
          epochs=10, batch_size=20, callbacks=[checkpointer], verbose=1)

In [ ]:
## 加载具有最好验证loss的模型

VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### 测试模型

In [ ]:
def dogorcat(file):
    bnf = model.predict(preprocess_input(path_to_tensor(file).astype('float32')/255))
    predict = VGG16_model.predict(bnf)
    return predict

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt       


img_path = os.path.join('test/128.jpg')
img = mpimg.imread(img_path,0)
plt.imshow(img)
plt.show()

predict = dogorcat(img_path)
dog_ratio = predict[0][0]*100
cat_ratio = predict[0][1]*100
print('狗：%.2f%%'% dog_ratio)
print('猫：%.2f%%'% cat_ratio)

In [18]:
# 获取测试数据集中每一个图像所预测的狗品种的index
dogorcat_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature,axis=0))) for feature in test_VGG16]

# 报告测试准确率
test_accuracy = 100*np.sum(np.array(dogorcat_predictions)==np.argmax(test_targets[:300], axis=1))/len(dogorcat_predictions)
print('Test accuracy: %.2f%%' % test_accuracy)

Test accuracy: 48.67%


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 

list = os.listdir('test')
f, axs = plt.subplots(2,4,figsize=(15,8))
for c in range(2):
    for r in range(4):
        img_path = os.path.join('test',list[c*2+r])
        predict = dogorcat(img_path)
        img = mpimg.imread(img_path,0)
        axs[c][r].set_xticks([])
        axs[c][r].set_yticks([])
        axs[c][r].imshow(img)
        if(np.argmax(predict[0])==0):
            s = 'dog' 
        else:
            s = 'cat'
        s= '%s:%.2f%%' % (s,max(predict[0])*100)
        axs[c][r].set_title(s)

In [ ]:
score = VGG16_model.evaluate